In [103]:
import pandas as pd
import numpy as np
import math
import os
import neurokit2 as nk
import nolds
from scipy import signal
import heartpy as hp

In [104]:
pd.options.mode.chained_assignment = None

In [105]:
where_PPG = 'C:/Users/citioplab/Downloads/TVGH002'

# 前處理 acc

In [106]:
# 消除基線飄移
def filt_ma(acc):
    newlist = [x for x in acc if math.isnan(x) == False]
    b, a = signal.butter(3, 0.1, btype="highpass", output="ba")
    ma_acc = signal.filtfilt(b, a, newlist)
    return ma_acc

# 讀一位病人[02]所有的資料

In [107]:
def time_imputation(df, column, filename):
    # 向下補值
    df[column].replace(0, np.nan, inplace=True)
    df[column].fillna(method='ffill', inplace=True)
    # 補開頭
    df[column].replace(np.nan, filename, inplace=True)
    # 統一格式
    df[column] = df[column].astype(str).str.rstrip('.0')
    df[column] = df[column].str.pad(width=14, side='right', fillchar='0')
    df[column] = pd.to_datetime(df[column])
    return df

In [108]:
# 讀取資料，並做成dataframe
no_list = ["02"]
day_dict = {}

for who in no_list:
    # goto = where_PPG + r'\TVGH0' + str(who) + "\\"
    goto = where_PPG + "\\"
    raw =  os.listdir(goto)
    
    for raw_data in raw:
        new_df = pd.DataFrame(columns=['Measure_time', 'X_acc', 'Y_acc', 'Z_acc'])

        if "raw" in raw_data.split(".")[0]: 
            filename = raw_data.split(".")[0].replace('_raw.xlsx', '')[:14]
            df = pd.read_csv(goto + raw_data ,header = None)
            df = df.dropna()

            # 取出csv中的欄位值
            measure_time = np.array(df[0].values.tolist()).T[0]
            x_acc = filt_ma(np.array(df[[4]].values.tolist()).T[0]) # 消除基線飄移
            y_acc = filt_ma(np.array(df[[5]].values.tolist()).T[0]) # 消除基線飄移
            z_acc = filt_ma(np.array(df[[6]].values.tolist()).T[0]) # 消除基線飄移

            # 做成dataframe
            part_df = pd.DataFrame({'Measure_time': df[0].tolist(), 'X_acc':x_acc, 'Y_acc': y_acc, 'Z_acc': z_acc})
            file_start_time = filename[:13] + str((int(filename[13]) - 1))
            part_df = time_imputation(part_df, 'Measure_time', file_start_time)
            
            day_dict[filename] = part_df


C:\Users\citioplab\AppData\Local\Temp\ipykernel_5756\669521990.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[column].fillna(method='ffill', inplace=True)
C:\Users\citioplab\AppData\Local\Temp\ipykernel_5756\669521990.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[column].fillna(method='ffill', inplace=True)
C:\Users\citioplab\AppData\Local\Temp\ipykernel_5756\669521990.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[column].fillna(method='ffill', inplace=True)
C:\Users\citioplab\AppData\Local\Temp\ipykernel_5756\669521990.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[column].fillna(method='ffill', inplac

In [109]:
# 確認每秒不超過64筆資料
for data in day_dict.values():
    period_counts_df = pd.DataFrame({'Measure_time': data['Measure_time'].value_counts().index, 'Count': data['Measure_time'].value_counts().values})
    sorted_counts_df = period_counts_df.sort_values(by='Measure_time').reset_index(drop=True)
    if len(sorted_counts_df.loc[sorted_counts_df['Count'] > 64, 'Measure_time']) > 0:
        print(sorted_counts_df.loc[sorted_counts_df['Count'] > 64, 'Measure_time'])

# 計算SPT

In [110]:
def five_sec_acc_rolling_med(data):
    # 取平均改以秒為單位
    X_avg = pd.DataFrame(data.groupby('Measure_time')['X_acc'].mean())
    Y_avg = pd.DataFrame(data.groupby('Measure_time')['Y_acc'].mean())
    Z_avg = pd.DataFrame(data.groupby('Measure_time')['Z_acc'].mean())
    df_sec = pd.DataFrame({'X_acc': X_avg['X_acc'], 'Y_acc': Y_avg['Y_acc'], 'Z_acc': Z_avg['Z_acc']})
    
    # Compute rolling medians for x, y, z coordinates
    rolling_medians_x = df_sec['X_acc'].rolling(window=5).median()
    rolling_medians_y = df_sec['Y_acc'].rolling(window=5).median()
    rolling_medians_z = df_sec['Z_acc'].rolling(window=5).median()

    # Combine rolling medians into a new DataFrame
    rolling_medians_df = pd.DataFrame({
        'rolling_median_x': rolling_medians_x,
        'rolling_median_y': rolling_medians_y,
        'rolling_median_z': rolling_medians_z
    })
    rolling_medians_df = rolling_medians_df.dropna()

    return rolling_medians_df

In [111]:
def calculate_angle(acc_x, acc_y, acc_z):
    deg = math.atan(acc_x / (math.sqrt(acc_y**2 + acc_z**2)))*180/math.pi
    return deg

In [112]:
def angle_diff(rolling_medians_df):
    rolling_medians_df['angle'] = rolling_medians_df.apply(lambda row: calculate_angle(row['rolling_median_x'], row['rolling_median_y'], row['rolling_median_z']), axis=1)
    
    # Compute 5 sec rolling average for angles
    rolling_avg_angle = pd.DataFrame(rolling_medians_df['angle'].rolling(window=5).mean())
    # rolling_avg_angle = rolling_avg_angle.dropna()

    # Compute the difference between consecutive angle averages
    rolling_avg_angle['angle_diff'] = rolling_avg_angle['angle'].diff()
    rolling_avg_angle['angle_diff'] = abs(rolling_avg_angle['angle_diff'])
    rolling_avg_angle = rolling_avg_angle.dropna()

    # 確認測量時間是連續的(無中斷，2019-04-13 15:36:29是中斷的)
    rolling_avg_angle = rolling_avg_angle.reset_index()
    rolling_avg_angle['time_diff'] = pd.to_timedelta(rolling_avg_angle['Measure_time'].diff())
    filtered_df = rolling_avg_angle[rolling_avg_angle['time_diff'] != pd.Timedelta('0 days 00:00:01')]
    # if filtered_df.shape[0] > 1:
    #     return "Time is not continuous"
    
    # Compute the 5 minutes rolling median for the angle differences
    rolling_median_5min_angle = pd.DataFrame({
        'Measure_time': rolling_avg_angle['Measure_time'],
        'rolling_median_angle': rolling_avg_angle['angle_diff'].rolling(window=300).median()
    })
    rolling_median_5min_angle = rolling_median_5min_angle.dropna()
    
    return rolling_median_5min_angle


In [113]:
def block_detection(angDiff, q, verbose):
    # 標註靜止的時間
    threshold = angDiff['rolling_median_angle'].quantile(q)
    angDiff['still'] = angDiff['rolling_median_angle'].apply(lambda x: 1 if x < threshold else 0)  
    if verbose:
        print(angDiff['still'].value_counts())
        
    # 找到所有的blocks
    block_start = []
    block_end = []
    pre_value = 0

    for idx, value in enumerate(angDiff['still']):
        if pre_value == 0 and value == 1:
            block_start.append(angDiff['Measure_time'].iloc[idx])
        elif pre_value == 1 and value == 0:
            block_end.append(angDiff['Measure_time'].iloc[idx-1])
        pre_value = value
    block_duration = [block_end[i] - block_start[i] for i in range(len(block_end))]
    if verbose:
        print(len(block_start), len(block_end), len(block_duration))
    blocks_df = pd.DataFrame({'block_start': block_start[:len(block_end)], 'block_end': block_end, 'duration':block_duration})
    
    # 留下持續超過30分鐘的blocks，並計算time gap
    spt_blocks = blocks_df[blocks_df['duration'] > pd.Timedelta('0 days 00:10:00')]
    time_gap_list = []
    for idx, value in enumerate(spt_blocks['block_end']):
        if idx < len(spt_blocks)-1:
            time_gap_list.append(spt_blocks['block_start'].iloc[idx+1] - spt_blocks['block_end'].iloc[idx])
        else:
            time_gap_list.append(np.nan)
    spt_blocks['time_gap'] = time_gap_list
    return spt_blocks

In [114]:
def turn_blocks_to_SPT(spt_blocks):
    # 整併間隔<60min的blocks
    SPT_start = 0
    SPT_end = 0
    SPT_duration = pd.Timedelta('0 days 00:00:00')
    temp_start = 0
    temp_end = 0
    temp_duration = pd.Timedelta('0 days 00:00:00')

    for idx, value in enumerate(spt_blocks['time_gap']):

        if value < pd.Timedelta('0 days 00:60:00'):
            if idx == 0:
                temp_start = spt_blocks['block_start'].iloc[idx]
            elif spt_blocks['time_gap'].iloc[idx-1] > pd.Timedelta('0 days 00:60:00'):
                temp_start = spt_blocks['block_start'].iloc[idx]

            temp_end = spt_blocks['block_end'].iloc[idx+1]
            temp_duration += spt_blocks['duration'].iloc[idx]

        else:
            if temp_duration > SPT_duration:
                SPT_duration = temp_duration
                SPT_start = temp_start
                SPT_end = temp_end
            temp_duration = pd.Timedelta('0 days 00:00:00')
            
    return SPT_start, SPT_end


# MAIN

In [115]:
for date in day_dict.keys():
    data = day_dict[date]
    
    rolling_medians_df = five_sec_acc_rolling_med(data)
    angDiff = angle_diff(rolling_medians_df)
    spt_blocks = block_detection(angDiff, q = 0.55,verbose=False)
    SPT_start, SPT_end = turn_blocks_to_SPT(spt_blocks)
        
    print(f"Date:{pd.to_datetime(date[:14])} ; SPT_start:{SPT_start} ; SPT_end:{SPT_end}")

Date:2019-04-02 03:29:45 ; SPT_start:2019-04-02 10:33:41 ; SPT_end:2019-04-02 14:04:00
Date:2019-04-08 18:48:35 ; SPT_start:2019-04-08 20:49:23 ; SPT_end:2019-04-09 02:12:50
Date:2019-04-11 16:45:01 ; SPT_start:2019-04-11 23:16:03 ; SPT_end:2019-04-12 05:36:20
Date:2019-04-12 15:59:26 ; SPT_start:2019-04-13 00:15:37 ; SPT_end:2019-04-13 07:31:03
Date:2019-04-13 15:36:29 ; SPT_start:2019-04-14 00:19:54 ; SPT_end:2019-04-14 08:47:40
Date:2019-04-14 15:01:47 ; SPT_start:2019-04-14 23:35:46 ; SPT_end:2019-04-15 05:19:58
Date:2019-04-15 16:11:12 ; SPT_start:2019-04-15 20:55:05 ; SPT_end:2019-04-16 01:36:08
Date:2019-04-16 15:54:35 ; SPT_start:2019-04-16 18:36:14 ; SPT_end:2019-04-17 00:05:28
Date:2019-04-17 18:04:11 ; SPT_start:2019-04-18 04:57:41 ; SPT_end:2019-04-18 13:57:57


In [116]:
for date in day_dict.keys():
    data = day_dict[date]
    
    rolling_medians_df = five_sec_acc_rolling_med(data)
    angDiff = angle_diff(rolling_medians_df)
    spt_blocks = block_detection(angDiff, q = 0.6,verbose=False)
    SPT_start, SPT_end = turn_blocks_to_SPT(spt_blocks)
        
    print(f"Date:{pd.to_datetime(date[:14])} ; SPT_start:{SPT_start} ; SPT_end:{SPT_end}")

Date:2019-04-02 03:29:45 ; SPT_start:2019-04-02 10:33:27 ; SPT_end:2019-04-02 14:04:15
Date:2019-04-08 18:48:35 ; SPT_start:2019-04-08 20:49:14 ; SPT_end:2019-04-09 02:14:10
Date:2019-04-11 16:45:01 ; SPT_start:2019-04-11 23:15:52 ; SPT_end:2019-04-12 05:36:23
Date:2019-04-12 15:59:26 ; SPT_start:2019-04-13 00:15:28 ; SPT_end:2019-04-13 07:49:26
Date:2019-04-13 15:36:29 ; SPT_start:2019-04-13 23:56:45 ; SPT_end:2019-04-14 08:55:54
Date:2019-04-14 15:01:47 ; SPT_start:2019-04-14 23:31:52 ; SPT_end:2019-04-15 05:21:04
Date:2019-04-15 16:11:12 ; SPT_start:2019-04-15 20:52:41 ; SPT_end:2019-04-16 01:36:16
Date:2019-04-16 15:54:35 ; SPT_start:2019-04-16 18:36:13 ; SPT_end:2019-04-17 00:06:44
Date:2019-04-17 18:04:11 ; SPT_start:2019-04-18 04:57:38 ; SPT_end:2019-04-18 13:57:58
